In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import numpy as np

np.random.seed(5)

class LossHistroy(keras.callbacks.Callback):
    def init(self):
        self.losses=[]
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

def seq2dataset(seq, window_size):
    dataset=[]
    for i in range(len(seq)-window_size):
        subset = seq[i:(i+window_size+1)]
        dataset.append([code2idx[item] for item in subset])
    return np.array(dataset)

code2idx = {'c4':0,'d4':1,'e4':2,'f4':3,'g4':4,'a4':5, 'b4':6,
            'c8':7,'d8':8,'e8':9,'f8':10,'g8':11,'a8':12,'b8':13}
idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
           7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

seq = ['g8','e8','e4','f8','d8','d4','c8','d8','e8','f8','g8','g8','g4',
      'g8','e8','e8','e8','f8','d8','d4','c8','e8','g8','g8','e8','e8','e4',
      'd8','d8','d8','d8','d8','e8','f4','e8','e8','e8','e8','e8','f8','g4',
      'g8','e8','e4','f8','d8','d4','c8','e8','e8','g8','g8','e8','e8','e4']

dataset = seq2dataset(seq, window_size=4)

print(dataset.shape)
print(dataset)

x_train = dataset[:,0:4]
y_train = dataset[:,4]


max_idx_value = 13

x_train = x_train /float(max_idx_value)

y_train = np_utils.to_categorical(y_train)

one_hot_vec_size = y_train.shape[1]

print("one hot encoding vector size is ", one_hot_vec_size)

# 모델 구성하기
model = Sequential()
model.add(Dense(128, input_dim=4, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(one_hot_vec_size, activation='softmax'))

# 모델 학습과정 설정하기
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = LossHistroy()
history.init()

# 모델 학습시키기
model.fit(x_train, y_train, epochs=2000, batch_size=10, verbose=2, callbacks=[history])

# 학습과정 살펴보기
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(history.losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legent(['train'], loc='upper left')
plt.show()

# 모델 평가하기
scores = model.evaluate(x_train, y_train)
print("%s %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 사용하기 - 1 스텝 예측
pred_count = 50

seq_out = ['g8','e8','e4','f8']
pred_out = model.predict(x_train)

for i in range(pred_count):
    inx = np.argmax(pred_out[i])
    seq_out.append(idx2code[idx])

print("one step prediction : ", seq_out)

# 전곡 예측
seq_in = ['g8', 'e8', 'e4', 'f8']
seq_out = seq_in
seq_in = [code2idx[it] / float(max_idx_value) for it in seq_in]

for i in range(pred_count):
    sample_in = np.array(seq_in)
    sample_in = np.reshape(sample_in, (1,4))
    pred_out = model.predict(sample_in)
    idx = np.argmax(pred_out)
    seq_out.append(idx2code[idx])
    seq_in.append(idx/float(max_idx_value))
    seq_in.pop(0)
print("full song prediction: " ,seq_out)



(51, 5)
[[11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  8]
 [ 8  1  7  8  9]
 [ 1  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 11]
 [ 9 10 11 11  4]
 [10 11 11  4 11]
 [11 11  4 11  9]
 [11  4 11  9  9]
 [ 4 11  9  9  9]
 [11  9  9  9 10]
 [ 9  9  9 10  8]
 [ 9  9 10  8  1]
 [ 9 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9 11]
 [ 1  7  9 11 11]
 [ 7  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]
 [11  9  9  2  8]
 [ 9  9  2  8  8]
 [ 9  2  8  8  8]
 [ 2  8  8  8  8]
 [ 8  8  8  8  8]
 [ 8  8  8  8  9]
 [ 8  8  8  9  3]
 [ 8  8  9  3  9]
 [ 8  9  3  9  9]
 [ 9  3  9  9  9]
 [ 3  9  9  9  9]
 [ 9  9  9  9  9]
 [ 9  9  9  9 10]
 [ 9  9  9 10  4]
 [ 9  9 10  4 11]
 [ 9 10  4 11  9]
 [10  4 11  9  2]
 [ 4 11  9  2 10]
 [11  9  2 10  8]
 [ 9  2 10  8  1]
 [ 2 10  8  1  7]
 [10  8  1  7  9]
 [ 8  1  7  9  9]
 [ 1  7  9  9 11]
 [ 7  9  9 11 11]
 [ 9  9 11 11  9]
 [ 9 11 11  9  9]
 [11 11  9  9  2]]
one hot encoding vector size is  12
Epoch 1/2000
 - 1s - loss: 2.4684 - a